## Import,all_cols,warnings,dtype

In [ ]:
import pandas as pd
#Getting python version
from platform import python_version
print(python_version())

#To see all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#To supress warnings
import warnings
warnings.filterwarnings('ignore')

#To check dtype of a column
from pandas.api.types import infer_dtype
#infer_dtype(df['col'])

- intersect
- multi-indexing problem
- pivot 
- list_compre
- merge
- column of list
- age from dob
- replace values

In [ ]:
#To check common values between two columns
np.intersect1d(df['col1'],df['col1']).size
df=d1[d1['col1'].isin(d2['col1'])]


# tackle multi-indexing problem
grps.columns = ['_'.join(col) for col in grps.columns.values]


#For grouping and counting in pivot.
data=df.groupby(['loan_account_no','loan_payment_type']).agg({'loan_payment_type': 'count'})\
.rename(columns={'loan_payment_type':'count'}).reset_index()\
.pivot_table(index='loan_account_no',columns='loan_payment_type',values='count').fillna(0).reset_index()


#Convert pivot values in %
data.apply(lambda x: (x/x.sum())*100, axis=1).apply(lambda x: round(x,2))


#List comprehension with 'and' and 'or'
df['Channel_2'] = ['Regular' if i=='Green' else 'Delayed' if i=='Orange' else \
                    'Not_paid_recent' if (i=='Red' and j <- 0.1) else 'Erratic' for i,j in zip(df['Channel'],df['irr'])]


#Merging two dfs and dropping common cols
dfNew = df.merge(data, left_index=True, right_index=True,
                 how='outer', suffixes=('', '_y'))
dfNew.drop(dfNew.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)


#Get values of a group in a list and then string
mydf=data.groupby('loan_account_no')['col_list'].apply(list).reset_index()
mydf['col_string']=[','.join(i) for i in mydf['col_list']]


#Getting age from dob
round(((pd.to_datetime("today").normalize()-pd.to_datetime(csr['dob'])).dt.days)/365)


#Replacing values
df['loan_purpose'].replace(dict.fromkeys(['a','b','c'], 'z'),inplace=True)


#Extracting only numbers from string
df1['csr_id_y']=df1['csr_id_y'].str.extract('(\d+)').astype('float')




## Binary_scoring

In [ ]:
#Binary scoring
def binary_scoring(df, col_list, trans_col, loan_col):
    """
    This functions takes transactional data and creates binary scoring monthly.
    input:
    ------
        df : input pandas dataframe(dataframe)
        col_list : list of columns for to group the data(month, loan account number)(list)
        trans_col : amount column name(str)
        loan_col : loan amount column name(str)
    output:
    -------
        binary_df : binary dataframe with score and loan number
    """
    
    #################################################filtering_amount####################################################
    temp_df = df.copy(deep=True) ## copying the data into another dataframe
    temp_df = temp_df.groupby(col_list)[trans_col].agg("sum").reset_index() ## grouping the data
    temp_df = temp_df[temp_df[trans_col] >= 100] ## getting amount count more than 100
    
    
    #################################################creating_binary_dataframe########################################## 
    ## getting binary dataframe using groupby
    binary_df = temp_df.groupby(col_list)[loan_col].agg("count").unstack().reset_index().transpose().reset_index()
    binary_df = binary_df.fillna(0) ## filling na with 0
    binary_df["total_trans"] = [0] * binary_df.shape[0] ## creating new total trans column
    for col in binary_df.columns[1:]:  ## imapping all column to integer
        binary_df[col] = binary_df[col].astype("int")
        binary_df["total_trans"] = binary_df["total_trans"] + binary_df[col]
        binary_df["total_trans"] = [0] * binary_df.shape[0] ## creating new total count column
    binary_df.columns = binary_df.iloc[0] ## making the top row as column
    binary_df = binary_df.iloc[1:,:]  ## selecting rows from 2nd one
    binary_df.rename(columns = {col_list[0] : loan_col}, inplace=True)  ## renaming column
    binary_df.columns = binary_df.columns.astype("string") ## columns to string
    
    
    #################################################incremening_Score####################################################
    inter_df = binary_df.iloc[:,1:].copy(deep=True) ## getting only one hot encoded data
    col_name = inter_df.columns ## getting column names
    
    ## for each month running loop for score
    for i in range(0,len(binary_df.columns[1:])-1):
        
        condition_one = inter_df[( inter_df[col_name[i]] ==0 ) & ( inter_df[col_name[i+1]] == 0 )] ## condition 1
        binary_df.loc[ condition_one.index, col_name[i+1] ] = 0
    
        condition_two = inter_df[ ( inter_df[col_name[i]] ==0 ) & ( inter_df[col_name[i+1]] == 1 )]  ## condition 2
        binary_df.loc[ condition_two.index, col_name[i+1] ] = 1
        
        condition_three = inter_df[ ( inter_df[col_name[i]] > 0 ) & ( inter_df[col_name[i+1]] == 0 )]  ## condition 3
        binary_df.loc[ condition_three.index, col_name[i+1] ] = 0
        
        condition_four = inter_df[ ( inter_df[col_name[i]] > 0) & ( inter_df[col_name[i+1]] == 1 ) ]  ## condition 4
        binary_df.loc[ condition_four.index, col_name[i+1] ] = binary_df.loc[ condition_four.index, col_name[i+1] ] + \
                                                               binary_df.loc[ condition_four.index, col_name[i] ]
    
    #################################################npv_score####################################################
    binary_df['npv'] = [0] * binary_df.shape[0] ## getting all score zero with npv column
    for idx, col in enumerate(col_name):  ## itreating data monthly and calculating score
        monthly_npv = binary_df[ col ] / ( np.exp( -( idx + 1 ) ) ) 
        if idx != 0: binary_df['npv'] = binary_df['npv'] + monthly_npv
        else: binary_df['npv'] = monthly_npv
    
    return binary_df

scored_df = binary_scoring(df1, ["month_id", "loan_account_no"], "trans_amount", "loan_account_no")

## Charts

In [ ]:
#pie chart
plt.figure(figsize=(15,15))
palette_color = seaborn.color_palette('bright')
plt.pie(a['error_description'], labels=a['index'], colors=palette_color,
         autopct='%.0f%%')
plt.show()

#bar chart
sns.barplot(data=axis10,y='index',x='error_description')
plt.xticks(rotation=90)
plt.show()

#heat-map
sns.heatmap(df.corr(),cmap ="YlGnBu",annot=True,linewidths = 0.1, vmax=.3, square=True)

#Scatter-plot
sns.scatterplot(data=df, x="freq_log", y="irr_log", hue="Clusters")

## Cohort analysis

In [ ]:
# A function that will parse the date Time based cohort:  1 day of month
def get_month(x): return dt.datetime(x.year, x.month, 1) 
# Create transaction_date column based on month and store in TransactionMonth
df1['TransactionMonth'] = df1['datetime'].apply(get_month) 
# Grouping by customer_id and select the InvoiceMonth value
grouping = df1.groupby('account_no')['TransactionMonth'] 
# Assigning a minimum InvoiceMonth value to the dataset
df1['CohortMonth'] = grouping.transform('min')

#grouping = df1.groupby(['account_no','month_id'])['month_id'] 
## Assigning a minimum InvoiceMonth value to the dataset
#df1['CohortMonth'] = grouping.transform('min')
## printing top 5 rows

df1['datetime']=pd.to_datetime(df1['datetime'])
df1['CohortMonth']=pd.to_datetime(df1['CohortMonth'])

def get_date_int(df1, column):
    year = df1[column].dt.year
    month = df1[column].dt.month
    day = df1[column].dt.day
    return year, month, day
# Getting the integers for date parts from the `InvoiceDay` column
transcation_year, transaction_month, _ = get_date_int(df1, 'datetime')
# Getting the integers for date parts from the `CohortDay` column
cohort_year, cohort_month, _ = get_date_int(df1, 'CohortMonth')

#  Get the  difference in years
years_diff = transcation_year - cohort_year
# Calculate difference in months
months_diff = transaction_month - cohort_month
""" Extract the difference in months from all previous values
 "+1" in addeded at the end so that first month is marked as 1 instead of 0 for easier interpretation. 
 """
df1['CohortIndex'] = years_diff * 12 + months_diff  + 1 

# Counting daily active user from each chort
grouping = df1.groupby(['CohortMonth', 'CohortIndex'])
# Counting number of unique customer Id's falling in each group of CohortMonth and CohortIndex
cohort_data = grouping['customer_id'].apply(pd.Series.nunique)
cohort_data = cohort_data.reset_index()
 # Assigning column names to the dataframe created above
cohort_counts = cohort_data.pivot(index='CohortMonth',
                                 columns ='CohortIndex',
                                 values = 'customer_id')
# Printing top 5 rows of Dataframe
cohort_data.head()

cohort_sizes = cohort_counts.iloc[:,0]
retention = cohort_counts
# Coverting the retention rate into percentage and Rounding off.
retention

#average_standard_cost.index = average_standard_cost.index.strftime('%Y-%m')
## Initialize the figure
#plt.figure(figsize=(16, 10))
## Adding a title
#plt.title('Average Standard Cost: Monthly Cohorts', fontsize = 14)
## Creating the heatmap
sns.heatmap(retention, annot = True,vmin = 0.0, vmax =20,cmap="YlGnBu", fmt='g')
plt.ylabel('Cohort Month')
plt.xlabel('Cohort Index')
plt.yticks( rotation='360')
plt.show()

#To get only NEW vs Existing customer
#create a dataframe contaning CustomerID and first purchase date
min_mon = df.groupby('account_no').month_id.min().reset_index()
min_mon.columns = ['account_no','minmon']
df = pd.merge(df, min_mon, on='account_no')

#create a column called User Type and assign Existing 
#if User's First Purchase Year Month before the selected Invoice Year Month
df['UserType'] = 'New'
df.loc[df['month_id']>df['minmon'],'UserType'] = 'Existing'

## Get month wise payment method used for each customer
pivot=pd.pivot_table(df,columns='month_id',index='account_no',values='mode',aggfunc='first').reset_index()

## Simple streak cal.

In [2]:
import pandas as pd
import numpy as np
#Getting sample df
np.random.seed(12)
seq = np.random.choice(['paid', 'not_paid'], size=100, p=[.70, .30])
streaks = pd.Series(seq, name='collection').to_frame()

#the loop
l=[]
curr_val = 0
for i in streaks['collection']:
    if i=='paid':
        curr_val+=1
        l.append(curr_val)
    else:
        curr_val=0
        l.append(curr_val)
streaks['streak_counter1']=l        

C:\Users\Anusha Priya\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## Cleaning experience data
- Removing 'years' and 'month' and also chnaging to numbers according to given month or year.
- Examples

'8 Months', '2 Years', '1 Year', '2.8 Years', '2.5 yrs', '3 Years',
       '1.2 Years', '3.4 Years', '1.8 Yrs', '1.3 yrs', '3.3 Years',
       '3.9 Years', '4 Years', '5 Years', '6 Years', '2.6 Years',
       '4.7 Years', '1 Yr', '8.3 Yrs', '1.6 Yrs', '1.6 Years', '8 Years',
       '2.4 Years', '1.9 Years', '9 Years', '1.4 Yrs', '4.10 Years',
       '1.1 Yrs', '10.6 Years', '3.10 Years', '3.5 Years', '9 Months',
       '10 Months', '1.3 Years', '3.8 Yrs', '4.2 Years', '2.10 Years'"

In [ ]:
mon = re.compile('mon',re.IGNORECASE)
yer = re.compile('y',re.IGNORECASE)
l=[]
for i in new['csr_exp']:
    if type(i)==int or type(i)==float:
        l.append(i)
    elif mon.search(i) != None:
        l.append(round(int(re.findall('\d+',i)[0])/12,2))
    elif yer.search(i) != None:
        l.append(re.findall(r'\d+(?:\.\d+)?',i)[0])
    else:
        l.append(np.nan)
new['csr_exp']=l       

## Geocoding

In [ ]:
#Reverse geocoding - getting address from lat,long
#!pip install geopandas
!pip install geopy
#import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="a")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=0)
location = reverse((25.207,88.581), language='en', exactly_one=True)
location.raw['address']

#To calculate distance between two coordinates and get coordinates from place name.
from geopy import distance
geocoder = Nominatim(user_agent='a')
loc1 = 'delhi'
loc2 = 'bangalore'
coor1 = geocoder.geocode(loc1)
coor2 = geocoder.geocode(loc2)
#print(coor1.latitude)
lat1,long1 = 24.1158,83.2948
lat2,long2 = 26.3361,80.003
p1=(lat1,long1)
p2=(lat2,long2)
print(distance.distance(p1,p2))